In [1]:
import argparse
import sys

import numpy as np
import torch
from moshi.models.loaders import CheckpointInfo
from moshi.models.tts import DEFAULT_DSM_TTS_REPO, DEFAULT_DSM_TTS_VOICE_REPO, TTSModel

from IPython.display import display, Audio

In [2]:
# Configuration
text = "Hey there! How are you? I had the craziest day today."
voice = "expresso/ex03-ex01_happy_001_channel1_334s.wav"
print(f"See https://huggingface.co/{DEFAULT_DSM_TTS_VOICE_REPO} for available voices.")

See https://huggingface.co/kyutai/tts-voices for available voices.


In [3]:
# Set everything up
checkpoint_info = CheckpointInfo.from_hf_repo(DEFAULT_DSM_TTS_REPO)
tts_model = TTSModel.from_checkpoint_info(
    checkpoint_info, n_q=32, temp=0.6, device=torch.device("cuda")
)

# If you want to make a dialog, you can pass more than one turn [text_speaker_1, text_speaker_2, text_2_speaker_1, ...]
entries = tts_model.prepare_script([text], padding_between=1)
voice_path = tts_model.get_voice_path(voice)
# CFG coef goes here because the model was trained with CFG distillation,
# so it's not _actually_ doing CFG at inference time.
# Also, if you are generating a dialog, you should have two voices in the list.
condition_attributes = tts_model.make_condition_attributes(
    [voice_path], cfg_coef=2.0
)

config.json: 0.00B [00:00, ?B/s]

dsm_tts_1e68beda@240.safetensors:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

tokenizer-e351c8d8-checkpoint125.safeten(…):   0%|          | 0.00/385M [00:00<?, ?B/s]

tokenizer_spm_8k_en_fr_audio.model:   0%|          | 0.00/120k [00:00<?, ?B/s]

expresso/ex03-ex01_happy_001_channel1_33(…):   0%|          | 0.00/256k [00:00<?, ?B/s]

In [4]:
print("Generating audio...")

pcms = []
def _on_frame(frame):
    print("Step", len(pcms), end="\r")
    if (frame != -1).all():
        pcm = tts_model.mimi.decode(frame[:, 1:, :]).cpu().numpy()
        pcms.append(np.clip(pcm[0, 0], -1, 1))

# You could also generate multiple audios at once by extending the following lists.
all_entries = [entries]
all_condition_attributes = [condition_attributes]
with tts_model.mimi.streaming(len(all_entries)):
    result = tts_model.generate(all_entries, all_condition_attributes, on_frame=_on_frame)

print("Done generating.")
audio = np.concatenate(pcms, axis=-1)

Generating audio...
Done generating.


In [5]:
display(
    Audio(audio, rate=tts_model.mimi.sample_rate, autoplay=True)
)